Customer Behaviour Analysis

Database Connection

In [1]:
!pip install mysql-connector-python

In [30]:
import mysql.connector as db
import pandas as pd

In [31]:
user = 'santhiya'
passcode = 'Kathir@143'
host = 'localhost'
db_name = 'sys'

In [32]:
db_connection = db.connect(

                  host = host,
                  user = user,
                  password = passcode,
                  database = db_name
)

In [33]:
db_connection

Converting CSV file into Dataframe

In [12]:
#Loading the CSV file into dataframe

df1 = pd.read_csv(r'D:\PROJECT1-BATCH2\CSV Files\customers.csv')

SQL Table creation

In [34]:
cursor = db_connection.cursor()

In [31]:
create_sql_query = """
create table customer_details(
    customer_id INT,
    customer_name VARCHAR(500),
    customer_email VARCHAR(500),
    gender VARCHAR(20),
    age INT,
    location_id INT
    );
"""
    

In [32]:
cursor.execute(create_sql_query)
db_connection.commit()

 Insert Data into MySQL Table 

In [35]:
# insert query
insert_query = '''
INSERT INTO customer_details (customer_id, customer_name, customer_email, gender,age,location_id)
VALUES (%s, %s, %s, %s, %s, %s)
'''

# Iterate over each row in the DataFrame and insert it into the SQL table
for index, row in df1.iterrows():
    cursor.execute(insert_query, tuple(row))
    db_connection.commit()

print("Data inserted successfully")

Data inserted successfully


In [36]:
#Same process repeating for other csv files

#Loading the CSV file into dataframe

df2 = pd.read_csv(r'D:\PROJECT1-BATCH2\CSV Files\customer_journey.csv')
df3 = pd.read_csv(r'D:\PROJECT1-BATCH2\CSV Files\customer_reviews.csv')
df4 = pd.read_csv(r'D:\PROJECT1-BATCH2\CSV Files\engagement_data.csv')
df5 = pd.read_csv(r'D:\PROJECT1-BATCH2\CSV Files\geography.csv')
df6 = pd.read_csv(r'D:\PROJECT1-BATCH2\CSV Files\products.csv')


In [37]:
#creating tables
create_sql_query_t2 = """
create table customer_journey(
    journey_id INT,
    customer_id INT,
    product_id INT,
    visit_date VARCHAR(20),
    stage VARCHAR(100),
    action VARCHAR(100),
    Duration INT
    );
"""

cursor.execute(create_sql_query_t2)
db_connection.commit()

In [41]:
#creating tables
create_sql_query_t3 = """
create table customer_review(
    review_id INT,
    customer_id INT,
    product_id INT,
    review_date VARCHAR(20),
    rating INT,
    review_text VARCHAR(500)
    );
"""

cursor.execute(create_sql_query_t3)
db_connection.commit()

In [42]:
#creating tables
create_sql_query_t4 = """
create table engagement_data(
    engagement_id INT,
    content_id INT,
    content_type VARCHAR(100),
    likes INT,
    engagement_date VARCHAR(100),
    Campaign_ID INT,
    product_id INT,
    views_clicks VARCHAR(100)
    );
"""

cursor.execute(create_sql_query_t4)
db_connection.commit()

In [43]:
#creating tables
create_sql_query_t5 = """
create table Geography_details(
    location_id INT,
    country VARCHAR(100),
    city VARCHAR(100)
    );
"""

cursor.execute(create_sql_query_t5)
db_connection.commit()

In [44]:
#creating tables
create_sql_query_t6 = """
create table product_details(
    product_id INT,
    product_name VARCHAR(500),
    category VARCHAR(100),
    price VARCHAR(100)
    );
"""

cursor.execute(create_sql_query_t6)
db_connection.commit()

In [48]:
#replacing null values into 0
df2['Duration'] = df2['Duration'].fillna(0)

In [49]:
insert_query_t2 = '''
INSERT INTO customer_journey (journey_id, customer_id, product_id, visit_date,stage,action,Duration)
VALUES (%s, %s, %s, %s, %s, %s, %s)
'''

# Iterate over each row in the DataFrame and insert it into the SQL table
for index, row in df2.iterrows():
    cursor.execute(insert_query_t2, tuple(row))
    db_connection.commit()

print("Data inserted successfully")

Data inserted successfully


In [50]:
insert_query_t3 = '''
INSERT INTO customer_review (review_id, customer_id,product_id,review_date,rating,review_text)
VALUES (%s, %s, %s, %s, %s, %s)
'''

# Iterate over each row in the DataFrame and insert it into the SQL table
for index, row in df3.iterrows():
    cursor.execute(insert_query_t3, tuple(row))
    db_connection.commit()

print("Data inserted successfully")

Data inserted successfully


In [53]:
insert_query_t4 = '''
INSERT INTO engagement_data (engagement_id,content_id,content_type,likes,engagement_date,Campaign_ID,product_id,views_clicks)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
'''

# Iterate over each row in the DataFrame and insert it into the SQL table
for index, row in df4.iterrows():
    cursor.execute(insert_query_t4, tuple(row))
    db_connection.commit()

print("Data inserted successfully")

Data inserted successfully


In [54]:
insert_query_t5 = '''
INSERT INTO Geography_details (location_id,country,city)
VALUES (%s, %s, %s)
'''

# Iterate over each row in the DataFrame and insert it into the SQL table
for index, row in df5.iterrows():
    cursor.execute(insert_query_t5, tuple(row))
    db_connection.commit()

print("Data inserted successfully")

Data inserted successfully


In [55]:
insert_query_t6 = '''
INSERT INTO product_details (product_id,product_name,category,price)
VALUES (%s, %s, %s, %s)
'''

# Iterate over each row in the DataFrame and insert it into the SQL table
for index, row in df6.iterrows():
    cursor.execute(insert_query_t6, tuple(row))
    db_connection.commit()

print("Data inserted successfully")

Data inserted successfully


Customer Journey & Engagement Analysis:

In [36]:
#dropping customers
cursor = db_connection.cursor()
sql = """
SELECT stage, 
       COUNT(DISTINCT customer_id) AS number_of_customers_at_stage,
       COUNT(DISTINCT CASE WHEN action = 'Drop-off' THEN customer_id END) AS number_of_dropped_off_customers
  FROM customer_journey
 GROUP BY 
       stage
 ORDER BY 
    number_of_dropped_off_customers DESC;
"""
cursor.execute(sql)
drop_data = cursor.fetchall()
cursor.column_names
drop_table = pd.DataFrame(drop_data, columns = cursor.column_names)

drop_table


,stage,number_of_customers_at_stage,number_of_dropped_off_customers
0,Checkout,19,14
1,Homepage,40,0
2,ProductPage,24,0


In [12]:
#actions leading to successful conversions
sql2 = """
WITH conversion_paths AS (
    SELECT 
          cj.customer_id,
          cj.action,
          ej.content_type
      FROM customer_journey cj
	      ,engagement_data ej
	 WHERE cj.product_id = ej.product_id
	   AND cj.action = 'Purchase'
)
SELECT 
      content_type
     ,COUNT(DISTINCT customer_id) AS num_customers
  FROM conversion_paths
 GROUP BY content_type;
 """
cursor.execute(sql2)
conversn_data = cursor.fetchall()
cursor.column_names
curvrsn_data_table = pd.DataFrame (conversn_data, columns = cursor.column_names)

curvrsn_data_table

,content_type,num_customers
0,blog,6
1,newsletter,6
2,SOCIALMEDIA,4
3,Video,6


In [14]:
#Average duration spend on each stages
sql3 = """
SELECT stage
      ,AVG(duration)
  FROM customer_journey
 GROUP BY stage;
 """
cursor.execute(sql3)
eng_dur_data = cursor.fetchall()
cursor.column_names
eng_dur_table = pd.DataFrame (eng_dur_data, columns = cursor.column_names)

eng_dur_table

,stage,AVG(duration)
0,Checkout,45.0500
1,ProductPage,182.7692
2,Homepage,160.5370


Customer Reviews Analysis:

In [37]:
#average rating for each products

sql4 = """
SELECT p.product_name
      ,AVG(cr.rating) AS rating
  FROM customer_review cr
      ,product_details p
 WHERE cr.product_id = p.product_id
 GROUP BY p.product_name
 ORDER BY rating DESC ;
 """
cursor.execute(sql4)
rating_data = cursor.fetchall()
cursor.column_names
rating_table = pd.DataFrame (rating_data, columns = cursor.column_names)

rating_table

,product_name,rating
0,Football Helmet,5.0000
1,Hockey Stick,4.4000
2,Volleyball,4.0000
3,Climbing Rope,4.0000
4,Running Shoes,4.0000
5,Ski Boots,4.0000
6,Boxing Gloves,4.0000
7,Soccer Ball,4.0000
8,Surfboard,3.8889
9,Tennis Racket,3.8571


In [18]:
#Review Analysis
sql5 = """
SELECT p.product_name,
       AVG(r.rating) AS average_rating,
       COUNT(DISTINCT r.customer_id) AS review_count,
       COUNT(DISTINCT cj.customer_id) AS purchase_count,
       AVG(cj.duration) AS average_duration_on_site
  FROM customer_review r
      ,product_details p
      ,customer_journey cj
 WHERE r.product_id = p.product_id
   AND r.customer_id = cj.customer_id 
 GROUP BY 
       p.product_name
 ORDER BY 
       purchase_count DESC;
"""
cursor.execute(sql5)
review_data = cursor.fetchall()
cursor.column_names
review_data_table = pd.DataFrame(review_data, columns = cursor.column_names)

review_data_table

,product_name,average_rating,review_count,purchase_count,average_duration_on_site
0,Surfboard,4.1818,8,8,158.2727
1,Fitness Tracker,3.4615,7,7,112.2308
2,Swim Goggles,3.4444,6,6,141.4444
3,Tennis Racket,3.7778,6,6,159.6667
4,Yoga Mat,3.5556,6,6,184.0000
5,Kayak,3.4444,5,5,113.2222
6,Ski Boots,4.0000,5,5,210.0000
7,Climbing Rope,4.5000,3,3,190.5000
8,Dumbbells,3.5000,3,3,142.5000
9,Running Shoes,4.5000,3,3,170.8333


Customer Reviews Analysis

In [38]:
#Rating review analysis
sql6 = """
SELECT pd.product_id,
       pd.product_name,
       AVG(cr.rating) AS avg_rating,
       COUNT(cr.review_id) AS total_reviews
  FROM product_details pd
      ,customer_review cr
 WHERE pd.product_id = cr.product_id
 GROUP BY
       pd.product_id, pd.product_name
 ORDER BY 
       avg_rating DESC;
"""
cursor.execute(sql6)
rating_data = cursor.fetchall()
cursor.column_names
rating_table = pd.DataFrame(rating_data , columns = cursor.column_names)

rating_table

,product_id,product_name,avg_rating,total_reviews
0,8,Football Helmet,5.0000,3
1,19,Hockey Stick,4.4000,5
2,18,Volleyball,4.0000,3
3,15,Climbing Rope,4.0000,6
4,1,Running Shoes,4.0000,4
5,11,Ski Boots,4.0000,6
6,20,Boxing Gloves,4.0000,2
7,5,Soccer Ball,4.0000,3
8,17,Surfboard,3.8889,9
9,6,Tennis Racket,3.8571,7


In [40]:
#Customer retention details

sql7 = """
WITH purchases AS (
    SELECT customer_id
          ,DATE_FORMAT(visit_date, '%Y-%m') AS purchase_month
      FROM customer_journey
),
first_purchase AS (
    SELECT customer_id,
           MIN(DATE_FORMAT(visit_date, '%Y-%m')) AS first_month
      FROM customer_journey
  GROUP BY customer_id
),
retention AS (
    SELECT p.customer_id
          ,p.purchase_month
          ,f.first_month
      FROM purchases p
          ,first_purchase f
     WHERE p.customer_id = f.customer_id
)
SELECT first_month
      ,purchase_month
      ,COUNT(DISTINCT customer_id) AS retained_customers
  FROM retention
GROUP BY first_month, purchase_month
HAVING retained_customers >=2
ORDER BY retained_customers DESC;
"""
cursor.execute(sql7)
retention_data = cursor.fetchall()
cursor.column_names
retention_data_table = pd.DataFrame(retention_data, columns = cursor.column_names)

retention_data_table

,first_month,purchase_month,retained_customers
0,2023-07,2023-07,5
1,2023-08,2023-08,5
2,2025-03,2025-03,5
3,2025-06,2025-06,5
4,2024-04,2024-04,4
5,2023-04,2023-04,3
6,2023-05,2023-05,3
7,2023-06,2023-06,3
8,2023-09,2023-09,3
9,2024-02,2024-02,3
